TODO: 

* setup streams using python snowflake connector

In [1]:
import snowflake.connector
import os

/Users/jordanpacho/Desktop/Sigma_Labs/Coursework/Deloton-Exercise-Bikes/.venv/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
USER = os.environ.get('USER')
ACCOUNT = os.environ.get('ACCOUNT')
PASSWORD = os.environ.get('PASSWORD')
WAREHOUSE= os.environ.get('WAREHOUSE')
DATABASE= os.environ.get('DATABASE')
BATCH_SCHEMA= os.environ.get('BATCH_SCHEMA')

In [3]:
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=BATCH_SCHEMA
)
cs = conn.cursor()

#### Making table streams

In [4]:
query = """
    CREATE STREAM rides_stream
    ON TABLE rides
"""
# cs.execute(query)

In [13]:
query_stream = """
    SELECT *
    FROM rides_stream
"""

cs.execute(query_stream).fetch_pandas_all()

,RIDE_ID,USER_ID,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE,METADATA$ACTION,METADATA$ISUPDATE,METADATA$ROW_ID
0,63,4612,"{'epoch': 1665071361, 'fraction': 0}",517,31165.999,60.282397,42.889749,39.756286,124.676983,INSERT,False,4b229c2dc4b3c081c60bc86c55bfaf4410578be2


In [14]:
query = """
    SELECT user_id
        FROM users
"""

cs.execute(query).fetch_pandas_all()

,USER_ID
0,4603
1,4604
2,4607
3,4608
4,4609
5,4610
6,4611
7,4612


In [15]:
query = """
    SELECT user_id
        FROM rides_stream
"""

cs.execute(query).fetch_pandas_all()

,USER_ID
0,4612


In [16]:
query = """
    SELECT users.user_id, rides_stream.ride_id
        FROM users
        RIGHT JOIN rides_stream
        ON users.user_id = rides_stream.user_id
"""

cs.execute(query).fetch_pandas_all()

,USER_ID,RIDE_ID
0,4612,63


In [9]:
query = """
    CREATE OR REPLACE TABLE DELATON.ZOOKEEPERS_BATCH_PRODUCTION.RECENT_RIDES (
        USER_ID NUMBER(38,0) NOT NULL,
        RIDE_ID NUMBER(38,0) NOT NULL,
        GENDER VARCHAR(6) NOT NULL,
        AGE NUMBER(38,0) NOT NULL,
        BEGIN_TIMESTAMP TIMESTAMP_NTZ(9) NOT NULL,
        TOTAL_DURATION_SEC NUMBER(38,0) NOT NULL,
        TOTAL_POWER FLOAT NOT NULL,
        MEAN_POWER FLOAT NOT NULL,
        MEAN_RESISTANCE FLOAT NOT NULL,
        MEAN_RPM FLOAT NOT NULL,
        MEAN_HEART_RATE FLOAT NOT NULL
    )
"""

# cs.execute(query)

In [17]:
query = """
     WITH recent_rides_data AS(

            WITH user_gender_dob AS (
            SELECT user_id, gender,
                DATEDIFF(hour,date_of_birth,GETDATE())/8766000000 AS age
                FROM users
            )
            
            SELECT ugd.user_id, rs.ride_id, ugd.gender, ugd.age, rs.begin_timestamp, rs.total_duration_sec, rs.total_power,
                rs.mean_power, rs.mean_resistance, rs.mean_rpm, rs.mean_heart_rate

                FROM user_gender_dob AS ugd
                RIGHT JOIN rides_stream AS rs
                    ON ugd.user_id = rs.user_id
                WHERE TO_DATE(rs.begin_timestamp) > DATEADD(HOUR, -12, GETDATE())
        )
        SELECT * FROM recent_rides_data


"""

cs.execute(query).fetch_pandas_all()

,USER_ID,RIDE_ID,GENDER,AGE,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE
0,4612,63,Female,54,"{'epoch': 1665071361, 'fraction': 0}",517,31165.999,60.282397,42.889749,39.756286,124.676983


In [76]:
query = """
    INSERT INTO recent_rides
        WITH recent_rides_data AS(

            WITH user_gender_dob AS (
            SELECT user_id, gender,
                DATEDIFF(hour,date_of_birth,GETDATE())/8766000000 AS age
                FROM users
            )
            
            SELECT ugd.user_id, rs.ride_id, ugd.gender, ugd.age, rs.begin_timestamp, rs.total_duration_sec, rs.total_power,
                rs.mean_power, rs.mean_resistance, rs.mean_rpm, rs.mean_heart_rate

                FROM user_gender_dob AS ugd
                RIGHT JOIN rides_stream AS rs
                    ON ugd.user_id = rs.user_id
                WHERE TO_DATE(rs.begin_timestamp) > DATEADD(HOUR, -12, GETDATE())
        )
        SELECT * FROM recent_rides_data
"""

cs.execute(query)

NotSupportedError: Unknown error

In [33]:
query = """
   SELECT * 
    FROM rides_stream
"""

cs.execute(query).fetch_pandas_all()

,RIDE_ID,USER_ID,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE,METADATA$ACTION,METADATA$ISUPDATE,METADATA$ROW_ID
0,64,4614,"{'epoch': 1665071881, 'fraction': 0}",520,29488.716,56.709069,41.446154,52.190385,90.188462,INSERT,False,1feba8c281886d3810c3573be5676cfdb82661e5
1,63,4612,"{'epoch': 1665071361, 'fraction': 0}",517,31165.999,60.282397,42.889749,39.756286,124.676983,INSERT,False,4b229c2dc4b3c081c60bc86c55bfaf4410578be2
2,65,4614,"{'epoch': 1665072404, 'fraction': 0}",520,23287.198,44.783073,41.461538,40.661538,89.717308,INSERT,False,2f44dd2dde24e548be63b226bfa8571c975dc4f2


In [31]:
query = """
   SELECT * 
    FROM recent_rides
"""

cs.execute(query).fetch_pandas_all()

,USER_ID,RIDE_ID,GENDER,AGE,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE
0,4612,61,Female,0,"{'epoch': 1665070319, 'fraction': 0}",518,36770.601,70.985716,42.899614,48.687259,111.440154
1,4612,60,Female,0,"{'epoch': 1665069796, 'fraction': 0}",520,36724.767,70.624552,43.026923,48.375000,120.215385
2,4612,62,Female,0,"{'epoch': 1665070840, 'fraction': 0}",519,30622.722,59.003318,42.901734,47.518304,97.281310


In [28]:
query = f"""
    CREATE TASK recent_ride_ingestion
        WAREHOUSE = {WAREHOUSE}
        SCHEDULE = '10 minute'
    AS
        INSERT INTO recent_rides
        WITH recent_rides_data AS(

            WITH user_gender_dob AS (
            SELECT user_id, gender,
                DATEDIFF(hour,date_of_birth,GETDATE())/8766000000 AS age
                FROM users
            )
            
            SELECT ugd.user_id, rs.ride_id, ugd.gender, ugd.age, rs.begin_timestamp, rs.total_duration_sec, rs.total_power,
                rs.mean_power, rs.mean_resistance, rs.mean_rpm, rs.mean_heart_rate

                FROM user_gender_dob AS ugd
                RIGHT JOIN rides_stream AS rs
                    ON ugd.user_id = rs.user_id
                WHERE TO_DATE(rs.begin_timestamp) > DATEADD(HOUR, -12, GETDATE())
        )
        SELECT * FROM recent_rides_data
"""

cs.execute(query)

In [26]:
query = """
    DROP TASK recent_ride_updater
"""

cs.execute(query)

In [27]:
query = """
    DROP TASK recent_ride_ingestion
"""

cs.execute(query)

In [25]:
query = """
    DROP TASK recent_ride_lagger
"""

cs.execute(query)

In [53]:
query = """
    DELETE FROM recent_rides
"""

cs.execute(query)

In [23]:
query = f"""
    CREATE TASK recent_ride_lagger
        WAREHOUSE = {WAREHOUSE}
        SCHEDULE = '15 minute'
        AFTER recent_ride_ingestion
    AS
        SELECT CURRENT_TIMESTAMP
"""

cs.execute(query)

In [30]:
query = f"""
    CREATE TASK recent_ride_updater
        WAREHOUSE = {WAREHOUSE}
        AFTER recent_ride_ingestion
    AS
        DELETE FROM recent_rides
            WHERE TO_DATE(begin_timestamp) < DATEADD(HOUR, -12, GETDATE())
"""

cs.execute(query)